In [1]:
import numpy as np
from dataclasses import dataclass
import math
from typing import Literal, List
import gsw 

In [2]:
class PressureCoefficients:
    C1: float
    C2: float
    C3: float
    D1: float
    D2: float
    T1: float
    T2: float
    T3: float
    T4: float
    T5: float
    Slope: float
    Offset: float
    AD590M: float
    AD590B: float
    
pres_coefs = PressureCoefficients()
pres_coefs.C1 = -4.212464e+004
pres_coefs.C2 = -1.972709e-001
pres_coefs.C3 = 1.365930e-002

pres_coefs.D1 = 3.604100e-002
pres_coefs.D2 = 0.000000e+000

pres_coefs.T1 = 3.046128e+001
pres_coefs.T2 = -3.791930e-004
pres_coefs.T3 = 4.202260e-006
pres_coefs.T4 = 3.386190e-009
pres_coefs.T5 = 0.000000e+000

pres_coefs.Slope = 1.00000000
pres_coefs.Offset = -0.20000
pres_coefs.AD590M = 1.279410e-002
pres_coefs.AD590B = -9.215740e+000

hexstrings = [
    "0CD08410767B80458F0CCA6F1107AE7D58A3987C41F0DEF2FFFFBE0000BB29512A089DB7004D638E5C91C465",
    "0CD0811076768045960CCA6F1107B27D48A3986C40F0DEF2FFFFBD0000BB29512A089DB7004D638F5C91C465",
    "0CD08710767B8045960CCA6D1107B17D48A3986C40F10EF0FFFFBE0000BB29512A089DB7004D63905D91C465",
    "0CD08710767A80458F0CCA6C1107AE7D58A3986C3FF16EE7FFFFBE0000BB29512A089DB7004D63915D91C465",
    "0CD08A10767680459E0CCA6F1107B27D58A3987C40F18EE3FFFFBE0000BB29512A089DB7004D63925D91C465",
    "0CD08410767A8045960CCA6C1107B17D58A3988C41F18EE3FFFFBE0000BB29512A089DB7004D63935D91C465",
    "0CD0811076798045960CCA6A1107B17D58A498AC41F18EE1FFFFBE0000BB29512A089DB7004D63945D91C465",
    "0CD08410767C80458F0CCA6A1107B07D48A398BC40F15ED4FFFFBE0000BB29512A089DB7004D63955D91C465",
    "0CD08710767A8045960CCA6F1107B17D48A498CC41F13ECEFFFFBE0000BB29512A089DB7004D63965D91C465",
    "0CD08110767980459D0CCA6F1107B07D48A398DC41F13ECEFFFFBD0000BB29512A089DB7004D63975D91C465"
]
bytes_list_array = [[hex_string[i:i+2] for i in range(0, len(hex_string), 2)] for hex_string in hexstrings]

In [3]:
def pressure_from_frequency(pressure_period: np.ndarray, temp_counts: np.ndarray, coefs: PressureCoefficients):
    """
    Berechnet den Druck [dbar] aus Digiquartz-Frequenz und Temperaturfrequenz (arrays erlaubt).

    :param freq_hz: gemessene Druckfrequenz [Hz]
    :param temp_freq_hz: gemessene Temperaturfrequenz [Hz]
    :param coefs: PressureCoefficients
    :return: Druck in dbar
    """

    U = coefs.AD590M * temp_counts + coefs.AD590B
    print("U:", U)
    C = coefs.C1 + coefs.C2 * U + coefs.C3 * U **2
    D = coefs.D1 + coefs.D2 * U
    T0 = (
        coefs.T1 
        + coefs.T2 * U 
        + coefs.T3 * U **2 
        + coefs.T4 * U **3 
        + coefs.T5 * U **4
    )
    
    T = pressure_period
    print("T:", T)
    pressure = C * (1 - (T0**2/T**2)) * (1 - D * (1 -(T0**2/T**2)))
    pressure = pressure * coefs.Slope + coefs.Offset
    return pressure


In [4]:
pressure_counts = []

for byte_list in bytes_list_array:
    count = int(byte_list[6], 16) * 256 + int(byte_list[7], 16) + int(byte_list[8], 16) / 256
    count = count
    pressure_counts.append(count)

pressure_counts = np.array(pressure_counts)

print(pressure_counts)
temperature_counts = []

for hex_string in hexstrings:
    temperature_value = int(hex_string[74:77],16)
    temperature_counts.append(temperature_value)

temperature_counts = np.array(temperature_counts)




press_periode = 1/ pressure_counts

press_periode = press_periode * 10**6
# Periode zwischen den Messungen
print(press_periode)

pressure = pressure_from_frequency(press_periode, temperature_counts, coefs=pres_coefs)


print("Druck in psi", pressure, "gewünscht: 1.667")
pressure = pressure * 0.689476 # Druck von psia zu db laut seabird

print("Druck in dbar:", pressure, "gewünscht: 1.149")

[32837.55859375 32837.5859375  32837.5859375  32837.55859375
 32837.6171875  32837.5859375  32837.5859375  32837.55859375
 32837.5859375  32837.61328125]
[30.45293386 30.4529085  30.4529085  30.45293386 30.45287952 30.4529085
 30.4529085  30.45293386 30.4529085  30.45288314]
U: [6.6233558 6.6233558 6.6233558 6.6233558 6.6233558 6.6233558 6.6233558
 6.6233558 6.6233558 6.6233558]
T: [30.45293386 30.4529085  30.4529085  30.45293386 30.45287952 30.4529085
 30.4529085  30.45293386 30.4529085  30.45288314]
Druck in psi [16.45657584 16.52676095 16.52676095 16.45657584 16.60697258 16.52676095
 16.52676095 16.45657584 16.52676095 16.59694612] gewünscht: 1.667
Druck in dbar: [11.34641408 11.39480503 11.39480503 11.34641408 11.45010903 11.39480503
 11.39480503 11.34641408 11.39480503 11.44319603] gewünscht: 1.149


In [5]:
[32837.55859375 32837.5859375  32837.5859375  32837.55859375
 32837.6171875  32837.5859375  32837.5859375  32837.55859375
 32837.5859375  32837.61328125]
[30.45293386 30.4529085  30.4529085  30.45293386 30.45287952 30.4529085
 30.4529085  30.45293386 30.4529085  30.45288314]
[6.6233558 6.6233558 6.6233558 6.6233558 6.6233558 6.6233558 6.6233558
 6.6233558 6.6233558 6.6233558]
Druck in psi [16.51868064 16.58886575 16.58886575 16.51868064 16.66907738 16.58886575
 16.58886575 16.51868064 16.58886575 16.65905092] gewünscht: 1.667
Druck in dbar: [11.38923385 11.4376248  11.4376248  11.38923385 11.4929288  11.4376248
 11.4376248  11.38923385 11.4376248  11.4860158 ] gewünscht: 1.149

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2046343579.py, line 1)